In [1]:
import numpy as np
import torch
import clip
from tqdm.notebook import tqdm
from pkg_resources import packaging

print("Torch version:", torch.__version__)

/root/anaconda3/envs/llm-motion/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch version: 2.1.0+cu118


In [2]:
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [20]:
from sentence_transformers import SentenceTransformer, util
from PIL import Image

#Load CLIP model
model = SentenceTransformer('clip-ViT-B-32')

config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 85.5kB/s]
README.md: 100%|██████████| 1.91k/1.91k [00:00<00:00, 2.45MB/s]
0_CLIPModel/config.json: 100%|██████████| 4.03k/4.03k [00:00<00:00, 1.56MB/s]
0_CLIPModel/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]
0_CLIPModel/preprocessor_config.json: 100%|██████████| 316/316 [00:00<00:00, 124kB/s]

0_CLIPModel/special_tokens_map.json: 100%|██████████| 389/389 [00:00<00:00, 142kB/s]

0_CLIPModel/tokenizer_config.json: 100%|██████████| 604/604 [00:00<00:00, 244kB/s]
0_CLIPModel/merges.txt: 100%|██████████| 525k/525k [00:00<00:00, 1.39MB/s]
0_CLIPModel/vocab.json: 100%|██████████| 961k/961k [00:00<00:00, 1.31MB/s]
pytorch_model.bin: 100%|██████████| 605M/605M [00:43<00:00, 13.9MB/s] 


In [44]:
#Encode text descriptions
motion_1 = 'a man kicks something or someone with his left leg'
motion_2 = 'the standing person kicks with their left foot before going back to their original stance.'
motion_3 = 'a person falls to the ground in a sitting motion and then pops back up in a standing position.'
motion_4 = 'a person standing loses balance falling to the right and recovers standing'
motion_1_emb = model.encode(motion_1)
motion_2_emb = model.encode(motion_2)
motion_3_emb = model.encode(motion_3)
motion_4_emb = model.encode(motion_4)

In [46]:
cos_sim = util.cos_sim(motion_1_emb, motion_4_emb)
cos_sim

tensor([[0.8207]])

In [47]:
cos_sim = util.cos_sim(motion_1_emb, motion_2_emb)
cos_sim

tensor([[0.8226]])

In [5]:
##### ---- Network ---- #####
clip_model, clip_preprocess = clip.load("ViT-B/32", device=torch.device('cuda'), jit=False)  # Must set jit=False for training
clip.model.convert_weights(clip_model)  # Actually this line is unnecessary since clip by default already on float16
clip_model.eval()
for p in clip_model.parameters():
    p.requires_grad = False

In [26]:
text = clip.tokenize(clip_text).cuda()
feat_clip_text = clip_model.encode_text(text).float()

In [28]:
feat_clip_text

tensor([[ 2.9938e-02, -2.9968e-02, -5.4736e-01, -1.2964e-01, -6.6772e-02,
         -2.8735e-01,  8.3374e-02, -4.2822e-01, -2.6343e-01,  4.5929e-02,
         -1.9983e-01,  1.1084e-01, -4.7852e-02, -1.9861e-01,  2.8711e-01,
          5.2277e-02,  5.9418e-02, -1.1681e-02, -2.0111e-02,  1.9971e-01,
         -4.4067e-01,  1.3318e-01,  2.5073e-01, -4.1504e-01,  3.9526e-01,
          2.3425e-01,  2.4744e-01,  3.8770e-01, -1.5857e-01, -2.2229e-01,
          2.8613e-01, -1.8933e-01,  2.9443e-01, -3.1982e-02, -8.0518e-01,
          4.5471e-02, -2.9980e-01, -9.8022e-02, -4.3793e-02,  9.1492e-02,
          3.8940e-02,  4.1602e-01,  4.7035e-03,  5.9032e-04,  3.0054e-01,
          9.0561e-03, -6.3721e-02, -2.7612e-01,  8.6914e-02,  1.0791e-01,
          1.8323e-01, -2.3840e-01, -1.2321e-02,  1.2109e-01,  5.2441e-01,
         -4.0625e-01,  9.3079e-02,  3.6206e-01, -3.0615e-01, -1.8481e-01,
         -5.3662e-01,  1.1615e-01,  5.2429e-02,  2.6660e-01, -8.2458e-02,
         -3.6719e-01,  4.4189e-02, -5.